# 행정동 / 법정동 명칭 통일 및 결합을 위한 전처리

## 기준(지도파일)을 바탕으로 행정동 / 법정동 결합
-> 법정동 기준으로 정리

- 일원2동 / 개포3동 -> 개포3동

- 강일동 -> 강일동

- 상일1동 / 상일2동

- 오류2동


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/new_project1/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re

import json
import geopandas as gpd

# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [ ]:
### 통계청 코드 / 법정동 코드 결합을 위한 준비

with open(f'{drive_path}data/seoul_json/hangjeongdong_서울특별시.geojson') as f:
    data = json.load(f)

# 필요한 열을 선택하여 DataFrame으로 변환
df = pd.DataFrame(data['features'])

In [ ]:
df['properties'][15]

{'OBJECTID': 16,
 'adm_nm': '서울특별시 종로구 청운효자동',
 'adm_cd': '1101072',
 'adm_cd2': '1111051500',
 'sgg': '11110',
 'sido': '11',
 'sidonm': '서울특별시',
 'sggnm': '종로구'}

In [ ]:
dong_code = []

for dong in df['properties']:
    dong_name = dong['adm_nm']
    tong_cd = dong['adm_cd']
    hang_cd = dong['adm_cd2']

    dong_code.append({'dong_name' : dong_name,
                      '통계청코드' : tong_cd,
                      '도로명코드' : hang_cd})

In [ ]:
df_code = pd.DataFrame(dong_code)
df_code.head()

,dong_name,통계청코드,도로명코드
0,서울특별시 종로구 사직동,1101053,1111053000
1,서울특별시 종로구 삼청동,1101054,1111054000
2,서울특별시 종로구 부암동,1101055,1111055000
3,서울특별시 종로구 평창동,1101056,1111056000
4,서울특별시 종로구 무악동,1101057,1111057000


In [ ]:
# 코드를 사용하는 다양한 코드가 존재
# 모든 코드에 대응하는 df 생성

df_code['통계청코드'] =  (df_code['통계청코드']).astype('int').astype('O')
df_code['통계청코드8'] = df_code['통계청코드'].astype(str) + '0'
df_code['통계청코드8'] =  (df_code['통계청코드8']).astype('int').astype('O')

df_code['도로명코드'] =  (df_code['도로명코드']).astype('int').astype('O')
df_code['도로명코드8'] = df_code['도로명코드'].apply(lambda x : str(x)[:-2])
df_code['도로명코드8'] =  (df_code['도로명코드8']).astype('int').astype('O')

df_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dong_name  425 non-null    object
 1   통계청코드      425 non-null    object
 2   도로명코드      425 non-null    object
 3   통계청코드8     425 non-null    object
 4   도로명코드8     425 non-null    object
dtypes: object(5)
memory usage: 16.7+ KB


In [ ]:
### 동이름 변경, 분동 관련 코드
### 프로젝트 발전시 향후 정리

# df_code[(df_code['dong_name'] == '서울특별시 강동구 상일동') | (df_code['dong_name'] == '서울특별시 강동구 강일동' ) | (df_code['dong_name'] == '서울특별시 강남구 일원2동')]

# df_code.loc[372, 'dong_name'] = '서울특별시 강남구 개포3동'
# df_code.loc[372, '통계청코드8'] = '11230511'
# df_code.loc[372, '도로명코드8'] = '11680675'

# df_code.loc[406, '통계청코드8'] = '11250750'
# df_code.loc[406, '도로명코드8'] = '11740515'

# df_code.loc[407, '통계청코드8'] = '11250760'
# df_code.loc[407, '도로명코드8'] = '11740526'

# df_code[(df_code['dong_name'] == '서울특별시 강동구 상일동') | (df_code['dong_name'] == '서울특별시 강동구 강일동' ) | (df_code['dong_name'] == '서울특별시 강남구 개포3동')]

In [ ]:
df_code['도로명코드8'] =  (df_code['도로명코드8']).astype('int').astype('O')

In [ ]:
df_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dong_name  425 non-null    object
 1   통계청코드      425 non-null    object
 2   도로명코드      425 non-null    object
 3   통계청코드8     425 non-null    object
 4   도로명코드8     425 non-null    object
dtypes: object(5)
memory usage: 16.7+ KB


In [ ]:
# 통계청 코드
tong_code = pd.read_excel(f'{drive_path}/data/region_code/통계청_읍면동코드.xlsx')
tong_code

,시도,중분류,시군구,소분류,읍면동
0,서울특별시,NaN,NaN,NaN,NaN
1,서울특별시,11010.0,종로구,NaN,NaN
2,서울특별시,11010.0,종로구,1101053.0,사직동
3,서울특별시,11010.0,종로구,1101054.0,삼청동
4,서울특별시,11010.0,종로구,1101055.0,부암동
...,...,...,...,...,...
3787,제주특별자치도,39020.0,서귀포시,3902058.0,서홍동
3788,제주특별자치도,39020.0,서귀포시,3902059.0,대륜동
3789,제주특별자치도,39020.0,서귀포시,3902060.0,대천동
3790,제주특별자치도,39020.0,서귀포시,3902061.0,중문동


In [ ]:
tong_code = tong_code[tong_code['시도'] == '서울특별시']
tong_code = tong_code[tong_code['읍면동'].notna()]
tong_code.reset_index(inplace = True, drop = True)
tong_code

,시도,중분류,시군구,소분류,읍면동
0,서울특별시,11010.0,종로구,1101053.0,사직동
1,서울특별시,11010.0,종로구,1101054.0,삼청동
2,서울특별시,11010.0,종로구,1101055.0,부암동
3,서울특별시,11010.0,종로구,1101056.0,평창동
4,서울특별시,11010.0,종로구,1101057.0,무악동
...,...,...,...,...,...
421,서울특별시,11250.0,강동구,1125070.0,둔촌1동
422,서울특별시,11250.0,강동구,1125071.0,둔촌2동
423,서울특별시,11250.0,강동구,1125072.0,암사1동
424,서울특별시,11250.0,강동구,1125073.0,천호2동


In [ ]:
tong_code.rename(columns = {'소분류' : '통계청코드'}, inplace = True)
tong_code['통계청코드'] =  (tong_code['통계청코드']).astype('int').astype('O')
tong_code

,시도,중분류,시군구,통계청코드,읍면동
0,서울특별시,11010.0,종로구,1101053,사직동
1,서울특별시,11010.0,종로구,1101054,삼청동
2,서울특별시,11010.0,종로구,1101055,부암동
3,서울특별시,11010.0,종로구,1101056,평창동
4,서울특별시,11010.0,종로구,1101057,무악동
...,...,...,...,...,...
421,서울특별시,11250.0,강동구,1125070,둔촌1동
422,서울특별시,11250.0,강동구,1125071,둔촌2동
423,서울특별시,11250.0,강동구,1125072,암사1동
424,서울특별시,11250.0,강동구,1125073,천호2동


In [ ]:
# 행안부코드
hang_code = pd.read_excel(f'{drive_path}data/region_code/행안부_읍면동코드.xlsx')
hang_code

,행정동코드,시도명,시군구명,읍면동명
0,11110515,서울특별시,종로구,청운효자동
1,11110530,서울특별시,종로구,사직동
2,11110540,서울특별시,종로구,삼청동
3,11110550,서울특별시,종로구,부암동
4,11110560,서울특별시,종로구,평창동
...,...,...,...,...
3581,50130580,제주특별자치도,서귀포시,서홍동
3582,50130590,제주특별자치도,서귀포시,대륜동
3583,50130600,제주특별자치도,서귀포시,대천동
3584,50130610,제주특별자치도,서귀포시,중문동


In [ ]:
hang_code = hang_code[hang_code['시도명']=='서울특별시']
hang_code

,행정동코드,시도명,시군구명,읍면동명
0,11110515,서울특별시,종로구,청운효자동
1,11110530,서울특별시,종로구,사직동
2,11110540,서울특별시,종로구,삼청동
3,11110550,서울특별시,종로구,부암동
4,11110560,서울특별시,종로구,평창동
...,...,...,...,...
421,11740650,서울특별시,강동구,성내제2동
422,11740660,서울특별시,강동구,성내제3동
423,11740685,서울특별시,강동구,길동
424,11740690,서울특별시,강동구,둔촌제1동


In [ ]:
hang_code.rename(columns = {'읍면동명' : '읍면동', '시군구명' : '시군구', '행정동코드' : '도로명코드'}, inplace = True)
hang_code['도로명코드'] =  (hang_code['도로명코드']).astype('int').astype('O')
hang_code.head()

,도로명코드,시도명,시군구,읍면동
0,11110515,서울특별시,종로구,청운효자동
1,11110530,서울특별시,종로구,사직동
2,11110540,서울특별시,종로구,삼청동
3,11110550,서울특별시,종로구,부암동
4,11110560,서울특별시,종로구,평창동


# SHP 파일과 결합


In [ ]:
dong_shp = gpd.read_file(f'{drive_path}data/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')
dong_shp.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1..."
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1..."
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1..."
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1..."
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1..."


In [ ]:
# CODE의 type이 int일떄만 결합가능
# 왜 인지 모르지만 'O', 'str'은 결합이 안된다.
dong_shp['ADM_CD'] =  (dong_shp['ADM_CD']).astype('int').astype(int)
df_code['통계청코드8'] =  (df_code['통계청코드8']).astype('int').astype(int)




In [ ]:
# 통계청 코드와의 결합
df_merge = dong_shp.merge(df_code,
                          how = 'left',
                          left_on = 'ADM_CD',
                          right_on = '통계청코드8')

df_merge.reset_index(drop = True, inplace = True)

In [ ]:
df_merge.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드,통계청코드8,도로명코드8
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053,1111053000,11010530.0,11110530
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054,1111054000,11010540.0,11110540
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055,1111055000,11010550.0,11110550
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056,1111056000,11010560.0,11110560
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074,1117068500,11030740.0,11170685


In [ ]:
df_merge[df_merge['dong_name'].isna()]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드,통계청코드8,도로명코드8
380,20230701,개포3동,11230511,"POLYGON ((963175.149 1944601.151, 963183.741 1...",NaN,NaN,NaN,NaN,NaN
417,20230701,강일동,11250750,"POLYGON ((971452.641 1953347.663, 971446.624 1...",NaN,NaN,NaN,NaN,NaN
418,20230701,상일1동,11250760,"POLYGON ((971190.564 1949669.365, 971193.744 1...",NaN,NaN,NaN,NaN,NaN
419,20230701,상일2동,11250770,"POLYGON ((971880.053 1951049.530, 971880.001 1...",NaN,NaN,NaN,NaN,NaN


In [ ]:
# 행안부 코드와 결합
df_merge2 = df_merge.merge(hang_code,
                                how = 'outer',
                                left_on = '도로명코드8',
                                right_on = '도로명코드')

df_merge2.reset_index(drop = True, inplace = True)

df_merge2.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드_x,통계청코드8,도로명코드8,도로명코드_y,시도명,시군구,읍면동
0,20230701,사직동,11010530.0,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053,1111053000,11010530.0,11110530,11110530,서울특별시,종로구,사직동
1,20230701,삼청동,11010540.0,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054,1111054000,11010540.0,11110540,11110540,서울특별시,종로구,삼청동
2,20230701,부암동,11010550.0,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055,1111055000,11010550.0,11110550,11110550,서울특별시,종로구,부암동
3,20230701,평창동,11010560.0,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056,1111056000,11010560.0,11110560,11110560,서울특별시,종로구,평창동
4,20230701,한남동,11030740.0,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074,1117068500,11030740.0,11170685,11170685,서울특별시,용산구,한남동


In [ ]:
df_merge2[df_merge2.isna().any(axis=1)]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드_x,통계청코드8,도로명코드8,도로명코드_y,시도명,시군구,읍면동
380,20230701,개포3동,11230511.0,"POLYGON ((963175.149 1944601.151, 963183.741 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381,20230701,강일동,11250750.0,"POLYGON ((971452.641 1953347.663, 971446.624 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,20230701,상일1동,11250760.0,"POLYGON ((971190.564 1949669.365, 971193.744 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383,20230701,상일2동,11250770.0,"POLYGON ((971880.053 1951049.530, 971880.001 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,11680675,서울특별시,강남구,개포3동
427,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,11740515,서울특별시,강동구,강일동
428,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,11740525,서울특별시,강동구,상일제1동
429,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,11740526,서울특별시,강동구,상일제2동


In [ ]:
df_merge2.rename(columns = {'도로명코드_x' : '도로명코드10' , '도로명코드_y' : '도로명코드'}, inplace = True)
df_merge2.drop(columns = ['BASE_DATE', 'ADM_NM', 'geometry'], inplace = True)
df_merge2.head()

,ADM_CD,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동
0,11010530.0,서울특별시 종로구 사직동,1101053,1111053000,11010530.0,11110530,11110530,서울특별시,종로구,사직동
1,11010540.0,서울특별시 종로구 삼청동,1101054,1111054000,11010540.0,11110540,11110540,서울특별시,종로구,삼청동
2,11010550.0,서울특별시 종로구 부암동,1101055,1111055000,11010550.0,11110550,11110550,서울특별시,종로구,부암동
3,11010560.0,서울특별시 종로구 평창동,1101056,1111056000,11010560.0,11110560,11110560,서울특별시,종로구,평창동
4,11030740.0,서울특별시 용산구 한남동,1103074,1117068500,11030740.0,11170685,11170685,서울특별시,용산구,한남동


In [ ]:
# # df_code.loc[372, 'dong_name'] = '서울특별시 강남구 개포3동'
# df_merge2.loc[380, '도로명코드'] = '11680675'

# df_merge2.loc[380, '시도명'] = '서울특별시'
# df_merge2.loc[380, '시군구'] = '강남구'
# df_merge2.loc[380, '읍면동'] = '개포3동'

# # df_code.loc[372, 'dong_name'] = '서울특별시 강남구 개포3동'
# df_merge2.loc[381, '도로명코드'] = '11680675'

# df_merge2.loc[381, '시도명'] = '서울특별시'
# df_merge2.loc[381, '시군구'] = '강동구'
# df_merge2.loc[381, '읍면동'] = '상일제2동'

# # df_code.loc[372, 'dong_name'] = '서울특별시 강남구 개포3동'
# df_merge2.loc[419, '도로명코드'] = '11740520'

# df_merge2.loc[419, '시도명'] = '서울특별시'
# df_merge2.loc[419, '시군구'] = '강동구'
# df_merge2.loc[419, '읍면동'] = '상일제1동'

### 분석에서 제외
# 이걸 해결할 방법이 있다면 나중에 바꿔서 실행
df_merge2.dropna(inplace = True)

In [ ]:
df_merge2['읍면동'] = df_merge2['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
df_merge2['읍면동'].unique()

array(['사직동', '삼청동', '부암동', '평창동', '한남동', '왕십리제2동', '마장동', '중림동', '옥수동',
       '장충동', '광희동', '사근동', '후암동', '용산2가동', '남영동', '원효로제2동', '효창동', '용문동',
       '이태원제1동', '금호1가동', '금호4가동', '성수1가제1동', '성수2가제1동', '송정동', '용답동',
       '정릉제4동', '길음제1동', '성북동', '삼선동', '동선동', '휘경제1동', '이문제1동', '이문제2동',
       '답십리제1동', '상봉제1동', '중화제1동', '신내1동', '월곡제2동', '자양제3동', '구의제2동',
       '수유1동', '삼양동', '송천동', '삼각산동', '우이동', '월계2동', '정릉제1동', '무악동',
       '종로56가동', '이화동', '창신제1동', '창신제2동', '을지로동', '수색동', '진관동', '불광제2동',
       '응암제3동', '천연동', '하계2동', '홍은제2동', '남가좌제1동', '염리동', '연남동', '성산제1동',
       '상계8동', '상계34동', '신정3동', '논현2동', '가양제1동', '발산제1동', '공항동', '방화제1동',
       '방화제2동', '방화제3동', '대치1동', '신도림동', '구로제1동', '구로제3동', '구로제4동',
       '고척제1동', '고척제2동', '시흥제1동', '북가좌제2동', '충현동', '북아현동', '신촌동', '시흥제5동',
       '목4동', '당산제2동', '양평제1동', '양평제2동', '신길제1동', '신길제6동', '신길제7동',
       '망원제2동', '대림제3동', '영등포본동', '영등포동', '도림동', '문래동', '노량진제2동', '상도제1동',
       '상도제2동', '상도제3동', '상도제4동', '대방동', '신대방제2동', '흑석동', '노량진제1

In [ ]:
df_merge2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 422 entries, 0 to 425
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADM_CD     422 non-null    float64
 1   dong_name  422 non-null    object 
 2   통계청코드      422 non-null    object 
 3   도로명코드10    422 non-null    object 
 4   통계청코드8     422 non-null    float64
 5   도로명코드8     422 non-null    object 
 6   도로명코드      422 non-null    object 
 7   시도명        422 non-null    object 
 8   시군구        422 non-null    object 
 9   읍면동        422 non-null    object 
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


In [ ]:
df_merge2.head()

,ADM_CD,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동
0,11010530.0,서울특별시 종로구 사직동,1101053,1111053000,11010530.0,11110530,11110530,서울특별시,종로구,사직동
1,11010540.0,서울특별시 종로구 삼청동,1101054,1111054000,11010540.0,11110540,11110540,서울특별시,종로구,삼청동
2,11010550.0,서울특별시 종로구 부암동,1101055,1111055000,11010550.0,11110550,11110550,서울특별시,종로구,부암동
3,11010560.0,서울특별시 종로구 평창동,1101056,1111056000,11010560.0,11110560,11110560,서울특별시,종로구,평창동
4,11030740.0,서울특별시 용산구 한남동,1103074,1117068500,11030740.0,11170685,11170685,서울특별시,용산구,한남동


In [ ]:
df_merge2['읍면동'] = df_merge2['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
df_merge2['읍면동'] = df_merge2['읍면동'].apply(lambda x: re.sub(r'\·', '', x))

In [ ]:
df_merge2.to_csv(f'{drive_path}final/data/region_code.csv', index = False)